In [1]:
import json
import opendssdirect as dss
import time
import math

In [2]:
#open conflict matrix
fcm = open("ConflictMatrix.json", "r")
ConflictMatrix = json.loads(fcm.read())

In [3]:
# alternatives vector dictionary
devices = list(ConflictMatrix["setpoints"].keys())
devices



['BatteryUnit:52',
 'BatteryUnit:65',
 'BatteryUnit:76',
 'RatioTapChanger:reg1a',
 'RatioTapChanger:reg2a',
 'RatioTapChanger:reg3a',
 'RatioTapChanger:reg3c',
 'RatioTapChanger:reg4a',
 'RatioTapChanger:reg4b',
 'RatioTapChanger:reg4c']

In [4]:
alt = {}
alt['a1'] = {}
alt['a1']['RatioTapChanger:reg1a'] =  -2
alt['a1']['RatioTapChanger:reg2a'] =  -1
alt['a1']['RatioTapChanger:reg3a'] =  4

alt['a2'] = {}
alt['a2']['RatioTapChanger:reg1a'] =  -1
alt['a2']['RatioTapChanger:reg2a'] =  -1
alt['a2']['RatioTapChanger:reg3a'] =  4

alt['a3'] = {}
alt['a3']['RatioTapChanger:reg1a'] =  -0
alt['a3']['RatioTapChanger:reg2a'] =  -1
alt['a3']['RatioTapChanger:reg3a'] =  4


In [5]:
dss.run_command('Redirect ./123Bus/Run_IEEE123Bus.dss')

''

In [6]:
dss.run_command('Compile ./123Bus/Run_IEEE123Bus.dss')

''

In [7]:
dss.Solution.Solve()

In [8]:
StartTime = time.process_time()

violations = False

bus_volt = dss.Circuit.AllBusMagPu()
print(min(bus_volt), max(bus_volt))
if min(bus_volt) < 0.95:
    violations = True
    print("violations!")
elif max(bus_volt) > 1.10:
    violations = True
    print("violations!")

sub = dss.Circuit.TotalPower()
sub_kva = math.sqrt((sub[0]*sub[0]) + (sub[1]*sub[1]))
if sub_kva > 5000 or sub_kva < -5000:
    violations = True
    
dss.Circuit.SetActiveClass('Transformer')
device = dss.Circuit.FirstElement()
while device:
    a = dss.CktElement.CurrentsMagAng()
    if a[0] > 668:
        violations = True
        
    device = dss.Circuit.NextElement()
print(time.process_time() - StartTime)
    

0.9799452732352611 1.0574482418074136
0.0025809380000001436


In [9]:
StartTime = time.process_time()

c_t = 0.02817 #transmission price $/kWh
c_r = 0.1090 #retail rate
c_n = 0.01786 #feed-in tariff
c_dies = 0.34 #diesel fuel cost
c_lng = 0.25 #lng fuel cost

e_t = 1.205 #grid co2 lb/kWh
e_dies = 2.44
e_lng = 0.97


bus_shunt_p = {}
total_load = 0
total_pv = 0
total_dg = 0
profit = 0
emissions = 0

sub = dss.Circuit.TotalPower()
p_sub = sub[0]

loss = dss.Circuit.Losses()
p_loss = loss[0]/1000

dss.Circuit.SetActiveClass('Load')
device = dss.Circuit.FirstElement()

while device:
    output = dss.CktElement.TotalPowers()
    bus = dss.CktElement.BusNames()
    bus_shunt_p[bus[0]] = output[0]
    total_load = total_load + output[0]
    device = dss.Circuit.NextElement()
    
dss.Circuit.SetActiveClass('PVSystem')
device = dss.Circuit.FirstElement()
while device:
    output = dss.CktElement.TotalPowers()
    bus = dss.CktElement.BusNames()
    total_pv = total_pv + output[0]
    if bus[0] in bus_shunt_p.keys():
        bus_shunt_p[bus[0]] = bus_shunt_p[bus[0]] + output[0]
    else:
        bus_shunt_p[bus[0]] = output[0]
    device = dss.Circuit.NextElement()
    
dss.Circuit.SetActiveClass('Generator')
device = dss.Circuit.FirstElement()
while device:
    output = dss.CktElement.TotalPowers()
    p_dg = output[0]
    total_dg = total_dg + p_dg
    if 'dies' in dss.CktElement.Name():
        profit = profit + p_dg*c_dies
        emissions = emissions - p_dg*e_dies
    elif 'lng' in dss.CktElement.Name():
        profit = profit + p_dg*c_lng
        emissions = emissions - p_dg*e_lng
    device = dss.Circuit.NextElement()

emissions = emissions - p_sub*e_t
profit = profit + p_sub*c_t
for bus in list(bus_shunt_p.keys()):
    p = bus_shunt_p[bus]
    if  p > 0:
        profit = profit + p*c_r
    else:
        profit = profit + p*c_n
        

    
print(time.process_time() - StartTime)
print(profit)
print(p_loss)
print(emissions)

0.0036853080000001093
38.21347104306829
48.600097801201144
2012.4441098550458


In [ ]:
bus_volt = dss.Circuit.AllBusMagPu()
print(min(bus_volt), max(bus_volt))
if min(bus_volt) < 0.95:
    print("violations!")
elif max(bus_volt) > 1.10:
    print("violations!")

In [ ]:
dss.Circuit.AllBusMagPu()

In [ ]:
dss.Transformers.AllNames()

In [ ]:
xfmr_id = dss.Transformers.First()
tap_vals = {}
while xfmr_id:
    tap_vals[dss.Transformers.Name()] = round((dss.Transformers.Tap()-1)*160)
    xfmr_id = dss.Transformers.Next()
    

In [ ]:
tap_vals

In [ ]:
xfmr_id = dss.Transformers.First()
print(dss.Transformers.NumTaps())
print(dss.Transformers.MinTap())
print(dss.Transformers.MaxTap())

In [ ]:
dss.Circuit.Losses()

In [ ]:
dss.Circuit.TotalPower()

In [ ]:
dss.Capacitors.AllNames()

In [ ]:
cap_id = dss.Capacitors.First()
cap_vals = {}
while cap_id:
    cap_vals[dss.Capacitors.Name()] = dss.Capacitors.States()
    cap_id = dss.Capacitors.Next()

In [ ]:
dss.Capacitors.

In [ ]:
StartTime = time.process_time()
load_id = dss.Loads.First()
load_vals = {}
total_load = 0
while load_id:
    load = dss.Loads.kW()
    load_vals[dss.Loads.Name()] = load
    load_id = dss.Loads.Next()
    total_load = total_load + load
print(time.process_time() - StartTime)

In [ ]:
dss.Ckt.Name()

In [ ]:
dss.run_command('Export Loads')

In [ ]:
dss.Circuit.SetActiveClass('Capacitor')
device = dss.Circuit.FirstElement()
total_load = 0
bus_shunt_p = {}
while device:
    output = dss.CktElement.VoltagesMagAng()
    print(output)
#     bus = dss.CktElement.BusNames()
#     bus_shunt_p[bus[0]] = output[0]
    device = dss.Circuit.NextElement()
# print(bus_shunt_p)

In [ ]:
dss.Circuit.SetActiveClass('Transformer')

dss.CktElement.Name()

In [ ]:
dss.Circuit.SetActiveClass('Transformer')
device = dss.Circuit.FirstElement()
total_load = 0
bus_shunt_p = {}
while device:
    s = dss.CktElement.TotalPowers()
    a = dss.CktElement.CurrentsMagAng()
    print(a[0])
    kva = math.sqrt((s[0]*s[0]) + (s[1]*s[1]))
    print(dss.CktElement.Name())
#     print(kva)
#     bus = dss.CktElement.BusNames()
#     bus_shunt_p[bus[0]] = output[0]
    device = dss.Circuit.NextElement()
# print(bus_shunt_p)

In [ ]:
a = dss.Circuit.SetActiveElement('PVSystem.')

In [ ]:
dss.CktElement.BusNames(

In [ ]:
dss.dss_lib.ActiveClass_Get_First()

In [ ]:
(dss.dss_lib.Bus_Get_Next())
ffi.string(dss.dss_lib.Bus_Get_Name())

In [ ]:
tap = 1
val = 1 + tap/160
StartTime = time.process_time()
dss.run_command('BatchEdit PVsystem..* irradiance=1')
dss.run_command('Transformer.reg4a.Taps=[1.0 %s]'%val)
dss.Solution.SolveNoControl()
print(time.process_time() - StartTime)


In [ ]:
from opendssdirect.utils import Iterator
import cffi

In [ ]:
[i() for i in Iterator(dss.Bus, "name")]

In [ ]:
from cffi import FFI
ffi = FFI()

In [ ]:
a = dss.Bus.AllPCEatBus()

In [ ]:
dss.Circuit.SetActiveElement(a[0])

In [ ]:
sub = dss.Circuit.TotalPower()
math.sqrt((sub[0]*sub[0]) + (sub[1]*sub[1]))

In [ ]:
sub

In [ ]:
import math